In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pickle
from tqdm import tqdm
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision


In [2]:
# 1. Set up CIFAR-10 Dataset with Transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = torchvision.datasets.CIFAR10(root='../data/train', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)

test_dataset = torchvision.datasets.CIFAR10(root='../data/test', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [5]:
# Model definition as before
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        print(f'Input range: min={x.min().item()}, max={x.max().item()}')
        x = self.pool(torch.relu(self.conv1(x)))
        print(f'After Conv1 + ReLU + Pool: min={x.min().item()}, max={x.max().item()}')
        x = self.pool(torch.relu(self.conv2(x)))
        print(f'After Conv2 + ReLU + Pool: min={x.min().item()}, max={x.max().item()}')
        x = x.view(-1, 64 * 8 * 8)
        x = torch.relu(self.fc1(x))
        print(f'After FC1 + ReLU: min={x.min().item()}, max={x.max().item()}')
        x = self.fc2(x)
        print(f'After FC2: min={x.min().item()}, max={x.max().item()}')
        return x

# Instantiate the model and test with a single batch to observe ranges
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)
model.load_state_dict(torch.load('./simple_cnn.pth'))
model.eval()

/tmp/ipykernel_37923/3357891942.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('./simple_cnn.pth'))


SimpleCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=4096, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [6]:
# Assuming validation data is loaded in validation_loader
validation_data = []
for images, labels in test_loader:
    validation_data.append((images, labels))

images, labels = next(iter(validation_data))
images, labels = images.to(device), labels.to(device)

with torch.no_grad():
    model(images)

Input range: min=-1.0, max=1.0
After Conv1 + ReLU + Pool: min=0.0, max=4.329313278198242
After Conv2 + ReLU + Pool: min=0.0, max=15.195003509521484
After FC1 + ReLU: min=0.0, max=58.971343994140625
After FC2: min=-88.17826843261719, max=70.5002670288086


In [7]:
conv1_weights = model.conv1.weight.data.cpu().numpy().flatten()[:5]
print("Sample Conv1 weights (first 5 in PyTorch):", conv1_weights)


Sample Conv1 weights (first 5 in PyTorch): [-0.154017    0.26767483 -0.03402724 -0.35736704 -0.40397918]
